In [123]:
import datetime

import pandas as pd
import numpy as np

import sqlite3
import os

import sys

sys.path.append("../")

In [124]:
%load_ext autoreload
%autoreload 2

from utils import SqliteUtils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [125]:
db_local_path = '../db.sqlite3'
sql = SqliteUtils(db_local_path)

sql.get_all_tables()

[('django_migrations',),
 ('sqlite_sequence',),
 ('auth_group_permissions',),
 ('auth_user_groups',),
 ('auth_user_user_permissions',),
 ('django_admin_log',),
 ('django_content_type',),
 ('auth_permission',),
 ('auth_group',),
 ('auth_user',),
 ('django_session',),
 ('weekly_planner_recipe',),
 ('weekly_planner_ingredients',),
 ('weekly_planner_recipe_fact',)]

In [126]:
# Drop tables
sql.drop_sqlite_table('weekly_planner_ingredients')
sql.drop_sqlite_table('weekly_planner_recipe')
sql.drop_sqlite_table('weekly_planner_recipe_fact')

In [127]:
# Check tables where actually loaded
sql.sqlite_to_pandas("""select * from weekly_planner_ingredients""")

""


In [128]:
# Check tables where actually loaded
sql.sqlite_to_pandas("""select * from weekly_planner_recipe_fact""")

""


# Get Data

In [129]:
# Import data
df = pd.read_excel('../data/processed/recipes_1.xlsx')

/Users/giovannibruner/opt/anaconda3/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:300: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [130]:
# Normalize quantity to monodose
df['mono_quantity'] = np.ceil(df['quantity'] / df['dose'])

In [131]:
# Add timestamp
from datetime import datetime
df['timestamp'] = datetime.now()

In [132]:
# Create ingredient ID

df['ingredient_id'] = ["{}_{}".format(x[0], x[1]).upper() for x in zip(df['ingredient'], df['unit_measure'])]


# Ingredients

In [133]:
# Check Existing Ingredients
new_ingredients = df[['ingredient', 'unit_measure', 'timestamp', 'ingredient_id']].drop_duplicates()

In [134]:
# Insert records
# Note - use cursor.description to check columns orders

conn = sqlite3.connect('../db.sqlite3')
c = conn.cursor()

for i in new_ingredients.itertuples():

    c.execute("""INSERT INTO weekly_planner_ingredients 
                ('Unit_measure', 'timestamp', 'Ingredient_id', 'Ingredient')
                VALUES ("{}", "{}", "{}", "{}")""".format(
        i.unit_measure, i.timestamp, i.ingredient_id, i.unit_measure))
    
conn.commit()
conn.close()

In [135]:
# Check tables where actually loaded
sql.sqlite_to_pandas("""select * from weekly_planner_ingredients""")

,unit_measure,timestamp,ingredient_id,ingredient
0,gr,2021-06-04 17:11:30.421980,AGLIO_GR,gr
1,q.b.,2021-06-04 17:11:30.421980,FARINA 00_Q.B.,q.b.
2,gr,2021-06-04 17:11:30.421980,MERLUZZO_GR,gr
3,q.b.,2021-06-04 17:11:30.421980,OLIO DI SEMI DI ARACHIDE_Q.B.,q.b.
4,gr,2021-06-04 17:11:30.421980,PANE_GR,gr
...,...,...,...,...
372,gr,2021-06-04 17:11:30.421980,ZAFFERANO_GR,gr
373,gr,2021-06-04 17:11:30.421980,ZUCCHERO DI CANNA_GR,gr
374,gr,2021-06-04 17:11:30.421980,POMODORINI_GR,gr
375,gr,2021-06-04 17:11:30.421980,SALSA BARBECUE_GR,gr


# Recipes

In [136]:
rec = df[['recipe_name', 'recipe_source', 'url', 'timestamp']]
rec = rec.drop_duplicates()

In [137]:
sql.get_columns_order('weekly_planner_recipe')

['recipe_name', 'recipe_source', 'url', 'timestamp']

In [138]:
conn = sqlite3.connect('../db.sqlite3')
c = conn.cursor()

for i in rec.itertuples():


    c.execute("""INSERT INTO weekly_planner_recipe 
                VALUES ("{}", "{}", "{}", "{}")""".format(i.recipe_name, 
                                                          i.recipe_source, 
                                                          i.url,
                                                          i.timestamp)) 

conn.commit()
conn.close()

In [139]:
# Check tables where actually loaded
sql.sqlite_to_pandas("""select * from weekly_planner_recipe""")

,recipe_name,recipe_source,url,timestamp
0,Polpette di pesce,GF,https://ricette.giallozafferano.it/Polpette-di...,2021-06-04 17:11:30.421980
1,Polpette di carne,GF,https://ricette.giallozafferano.it/Polpette-di...,2021-06-04 17:11:30.421980
2,Clafoutis salato,GF,https://ricette.giallozafferano.it/Clafoutis-s...,2021-06-04 17:11:30.421980
3,Polpette di pollo all'arancia,GF,https://ricette.giallozafferano.it/Polpette-di...,2021-06-04 17:11:30.421980
4,Polpette di zucchine alla pizzaiola,GF,https://ricette.giallozafferano.it/Polpette-di...,2021-06-04 17:11:30.421980
...,...,...,...,...
232,Merluzzo in foglia di porro,GF,https://ricette.giallozafferano.it/Merluzzo-in...,2021-06-04 17:11:30.421980
233,Cosce di pollo al forno,GF,https://ricette.giallozafferano.it/Cosce-di-po...,2021-06-04 17:11:30.421980
234,Tagliata alle erbe aromatiche,GF,https://ricette.giallozafferano.it/Tagliata-al...,2021-06-04 17:11:30.421980
235,Omelette di verdure,GF,https://ricette.giallozafferano.it/Omelette-di...,2021-06-04 17:11:30.421980


# Recipe Facts

In [140]:
sql.get_columns_order('weekly_planner_recipe_fact')

['id',
 'unit_measure',
 'quantity',
 'timestamp',
 'recipe_name',
 'ingredient_id',
 'ingredient']

In [141]:
facts = df[["recipe_name", "ingredient", "unit_measure", "mono_quantity", "timestamp", "ingredient_id"]]
facts = facts.drop_duplicates()

facts.head()

,recipe_name,ingredient,unit_measure,mono_quantity,timestamp,ingredient_id
0,Polpette di pesce,Aglio,gr,1.0,2021-06-04 17:11:30.421980,AGLIO_GR
1,Polpette di pesce,Farina 00,q.b.,NaN,2021-06-04 17:11:30.421980,FARINA 00_Q.B.
2,Polpette di pesce,Merluzzo,gr,88.0,2021-06-04 17:11:30.421980,MERLUZZO_GR
3,Polpette di pesce,Olio di semi di arachide,q.b.,NaN,2021-06-04 17:11:30.421980,OLIO DI SEMI DI ARACHIDE_Q.B.
4,Polpette di pesce,Pane,gr,13.0,2021-06-04 17:11:30.421980,PANE_GR


In [142]:
conn = sqlite3.connect('../db.sqlite3')
c = conn.cursor()

for i in facts.itertuples():

    c.execute("""INSERT INTO weekly_planner_recipe_fact 
                (unit_measure, quantity, timestamp, recipe_name, ingredient_id, ingredient)
                VALUES ("{}", "{}", "{}", "{}", "{}", "{}")""".format(i.unit_measure, 
                                                          i.mono_quantity, 
                                                          i.timestamp,
                                                          i.recipe_name,
                                                          i.ingredient_id,
                                                          i.ingredient
                                                         )) 

conn.commit()
conn.close()

In [143]:
# Check tables where actually loaded
sql.sqlite_to_pandas("""select * from weekly_planner_recipe_fact""")

,id,unit_measure,quantity,timestamp,recipe_name,ingredient_id,ingredient
0,8653,gr,1,2021-06-04 17:11:30.421980,Polpette di pesce,AGLIO_GR,Aglio
1,8654,q.b.,nan,2021-06-04 17:11:30.421980,Polpette di pesce,FARINA 00_Q.B.,Farina 00
2,8655,gr,88,2021-06-04 17:11:30.421980,Polpette di pesce,MERLUZZO_GR,Merluzzo
3,8656,q.b.,nan,2021-06-04 17:11:30.421980,Polpette di pesce,OLIO DI SEMI DI ARACHIDE_Q.B.,Olio di semi di arachide
4,8657,gr,13,2021-06-04 17:11:30.421980,Polpette di pesce,PANE_GR,Pane
...,...,...,...,...,...,...,...
2873,11526,gr,13,2021-06-04 17:11:30.421980,Omelette di verdure,SPINACI_GR,Spinaci
2874,11527,q.b.,nan,2021-06-04 17:11:30.421980,Omelette di verdure,TABASCO®_Q.B.,Tabasco®
2875,11528,gr,60,2021-06-04 17:11:30.421980,Omelette (ricetta base),UOVA_GR,Uova
2876,11529,gr,60,2021-06-04 17:11:30.421980,Omelette di verdure,UOVA_GR,Uova
